In [ ]:
import tensorflow as tf

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._experimental.autosegmentation import unet

In [ ]:
model = unet.unet(grid_size=64, output_channels=3)
# tf.keras.utils.plot_model(model, show_shapes=True)